In [17]:
import autogen
from typing import Literal
from typing_extensions import Annotated

local_llm_config={
    "config_list": [
        {
            "model": "NotRequired", # Loaded with LiteLLM command
            "api_key": "NotRequired", # Not needed
            "base_url": "http://0.0.0.0:4000"  # Your LiteLLM URL
        }
    ],
    "cache_seed": None # Turns off caching, useful for testing different models
}

# Create the agent and include examples of the function calling JSON in the prompt
# to help guide the model
chatbot = autogen.AssistantAgent(
    name="chatbot",
    system_message="""For currency exchange tasks,
        only use the functions you have been provided with.
        Output 'TERMINATE' when an answer has been provided.
        Do not include the function name or result in the JSON.
        Example of the return JSON is:
        {
            "parameter_1_name": 100.00,
            "parameter_2_name": "ABC",
            "parameter_3_name": "DEF",
        }.
        Another example of the return JSON is:
        {
            "parameter_1_name": "GHI",
            "parameter_2_name": "ABC",
            "parameter_3_name": "DEF",
            "parameter_4_name": 123.00,
        }. """,

    llm_config=local_llm_config,
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and "TERMINATE" in x.get("content", ""),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1,
    code_execution_config=False
)


CurrencySymbol = Literal["USD", "EUR"]

# Define our function that we expect to call
def exchange_rate(base_currency: CurrencySymbol, quote_currency: CurrencySymbol) -> float:
    if base_currency == quote_currency:
        return 1.0
    elif base_currency == "USD" and quote_currency == "EUR":
        return 1 / 1.1
    elif base_currency == "EUR" and quote_currency == "USD":
        return 1.1
    else:
        raise ValueError(f"Unknown currencies {base_currency}, {quote_currency}")

# Register the function with the agent
@user_proxy.register_for_execution()
@chatbot.register_for_llm(description="Currency exchange calculator.")
def currency_calculator(
    base_amount: Annotated[float, "Amount of currency in base_currency"],
    base_currency: Annotated[CurrencySymbol, "Base currency"] = "USD",
    quote_currency: Annotated[CurrencySymbol, "Quote currency"] = "EUR",
) -> str:
    quote_amount = exchange_rate(base_currency, quote_currency) * base_amount
    return f"{format(quote_amount, '.2f')} {quote_currency}"

# start the conversation
res = user_proxy.initiate_chat(
    chatbot,
    message="How much is 123.45 EUR in USD?",
    summary_method="reflection_with_llm",
)

user_proxy (to chatbot):

How much is 123.45 EUR in USD?

--------------------------------------------------------------------------------
[autogen.oai.client: 07-11 18:34:58] {315} WARNING - Model ollama_chat/llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
chatbot (to user_proxy):

***** Suggested tool call (call_7be26e33-1b1e-4fd2-96cb-dd1d20c4f235): currency_calculator *****
Arguments: 
{"base_amount": 123.45, "base_currency": "EUR", "quote_currency": "USD"}
************************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION currency_calculator...


[runtime logging] log_function_use: autogen logger is None


user_proxy (to chatbot):

user_proxy (to chatbot):

***** Response from calling tool (call_7be26e33-1b1e-4fd2-96cb-dd1d20c4f235) *****
135.80 USD
**********************************************************************************

--------------------------------------------------------------------------------
[autogen.oai.client: 07-11 18:34:59] {315} WARNING - Model ollama_chat/llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
chatbot (to user_proxy):

***** Suggested tool call (call_d3837222-1c06-4ea4-8528-ecd13084dadb): currency_calculator *****
Arguments: 
{"base_amount": 123.45, "base_currency": "EUR"}
************************************************************************************************

--------------------------------------------------------------------------------
[autogen.oai.client: 07-11 18:35:01] {315} WARNING - Model ollama_chat/llama3 is not found. The

In [ ]:
# from typing import Annotated, Literal

# Operator = Literal["+", "-", "*", "/"]


# def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> int:
#     if operator == "+":
#         return a + b
#     elif operator == "-":
#         return a - b
#     elif operator == "*":
#         return a * b
#     elif operator == "/":
#         return int(a / b)
#     else:
#         raise ValueError("Invalid operator")

In [ ]:
# # Register the tool signature with the assistant agent.
# helper.register_for_llm(name="calculator", description="A calculator, use this tool to do mathematical operation")(
#     calculator
# )

# # Register the tool function with the user proxy agent.
# agent.register_for_execution(name="calculator")(calculator)